In [3]:
pip install altair


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.23.1-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
   ---------------------------------------- 0.0/313.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/313.1 kB ? eta -:--:--
   ------- ------------------------------- 61.4/313.1 kB 812.7 kB/s eta 0:00:01
   ------------------ --------------------- 143.4/313.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------- ----------- 225.3/313.1 kB 1.4 MB/s eta 0:00:01
   ------------------------------------- -- 297.0/313.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 313.1/313.1 kB 1.4 MB/s et

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
import altair as alt

In [5]:
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

Kode di atas memiliki dua buah sintaks yang berperan sebagai ground rules ketika menggunakan Altair sebagai library visualisasi data. Mari kita jabarkan satu per satu.

1. alt.data_transformers.enable('default', max_rows=None): baris ini mengatur transformer data Altair untuk digunakan dalam notebook. Transformer data ini mengubah data mentah menjadi format yang dapat digunakan oleh Altair untuk membuat visualisasi. Dalam baris ini, kita mengatur transformer data menjadi 'default', biasanya parameter ini dapat mengubah data menjadi format yang cocok untuk visualisasi. Pengaturan max_rows=None berperan untuk menghilangkan batasan baris data yang dapat diplot. Ini bermanfaat jika kita memiliki dataset besar yang ingin kita visualisasikan sepenuhnya.
2. alt.renderers.enable('colab'): baris ini mengatur Altair agar dapat digunakan dalam Google Colab. Renderer adalah komponen yang bertanggung jawab untuk menampilkan visualisasi di lingkungan tempat kita bekerja, seperti browser atau notebook. Dalam baris ini, kita mengatur renderer menjadi 'colab' yang cocok untuk penggunaan Altair di Google Colab. Dengan mengatur renderer ini, visualisasi yang kita buat dengan Altair akan ditampilkan secara langsung di Google Colab.

In [7]:
# melaku,an downlaod data
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile
 
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [15]:
# Load masing masing dataset (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')
 
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# Genre pada setiap film bersifat biner.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = ['movie_id', 'title', 'release_date', "video_release_date", "imdb_url"] + genre_cols
movies = pd.read_csv('ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

In [16]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [17]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [14]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [23]:
#melakukan rekap
print('Jumlah data film: ', len(movies.movie_id.unique()))
print('Jumlah data pengunjung: ', len(users.user_id.unique()))
print('Jumlah data penayangan: ', len(ratings.user_id))

Jumlah data film:  1682
Jumlah data pengunjung:  943
Jumlah data penayangan:  100000


# eksplorasi data


In [27]:
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

TypeError: unsupported operand type(s) for -: 'str' and 'int'